### Init

In [1]:
!pip install anvil-uplink
!apt-get install xattr > /dev/null

import warnings
warnings.filterwarnings('ignore')
import numpy as np
from copy import deepcopy
import pandas as pd
import anvil.server

anvil.server.connect("server_LU6IJLIHIQMEDVFDQDGQHP4L-NTELYCTLUUWNSOLE")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=48ab988d2a4979da9d4b7abca4277e1822846bd30ffe268b9f0dd440f8eb674c
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


### Classes

In [2]:
class Path(object):
    def __init__(self, path, graph):
        assert type(path) is list
        self._path = path
        self._graph = graph
        self._num_city = len(graph)
        self._value = self._get_value()

    def path(self):
        return self._path

    def value(self):
        return self._value

    def _get_value(self):
        '''Returns the value of genetic code
           Value is a number (integer or floating point).'''
        assert type(self._path) is list

        if self._path[0] != self._path[-1] or len(self._path) != self._num_city + 1:
            return 1e-9

        cost = 0
        for idx, city in enumerate(self._path):
            if idx == len(self._path) - 1:
                break
            next_city = self._path[idx + 1]
            cost += self._graph[city][next_city]

        return 1/cost

    def __gt__(self, other):
        return self._value > other.value()

    def __add__(self, other):
        """
        'Crossover method' for genetic search. It should return a new genetic
         code that is the 'mix' of father and mother.
        """
        assert type(other) is Path

        half_path1 = deepcopy(self._path[:len(self._path)//2])
        half_path2 = deepcopy(other.path())
        half_path2 = list(filter(lambda city: city not in half_path1, half_path2))

        return Path(half_path1 + half_path2, self._graph)

    def mutate(self):
        middle = len(self._path)//2
        pos_1 = np.random.randint(0, middle)
        pos_2 = np.random.randint(middle, len(self._path))
        self._path[pos_1], self._path[pos_2] = self._path[pos_2], self._path[pos_1]
        self._value = self._get_value()

In [3]:
class GeneticSearch(object):
    def __init__(self, graph):
        assert type(graph) is dict
        self._graph = graph
        self._num_city = len(graph)

    def _get_random_path(self):
        """
        generate random genetic code
        """
        cities = list(self._graph)
        city_probs = np.ones(self._num_city) / self._num_city
        path = np.random.choice(cities, self._num_city, p=city_probs, replace=False)
        end_city = np.random.choice(cities, 1, p=city_probs)
        return path.tolist() + end_city.tolist()

    def _get_random_parents(self, population):
        population_values = [path.value() for path in population]
        total_values = sum(population_values)
        '''probabilities of each individual, each represent how likely that
           individual is chosen to become a parent'''
        population_probs = [value/total_values for value in population_values]
        return np.random.choice(population, 2, p=population_probs, replace=False)

    def _population_expander(self, population_size, mutation_chance):
        def expander(population):
            new_generation = []
            for _ in range(population_size):
                parents = self._get_random_parents(population)
                father, mother = parents.tolist()
                child = father + mother

                '''random if a child is likely to be mutated'''
                is_mutant = np.random.choice([True, False], 1, p=[mutation_chance, 1-mutation_chance])[0]
                if is_mutant:
                    child.mutate()
                new_generation.append(child)

            population += new_generation #add new generation to the current population
            population_values = [path.value() for path in population]
            total_values = sum(population_values)
            population_probs = [value/total_values for value in population_values]

            '''randomly choose individuals to fit new population'''
            population = np.random.choice(population, population_size, p=population_probs, replace=False)
            return population.tolist()

        return expander

    def search(self, num_generation=100, population_size=100, mutation_chance=0.1, patience=5):
        assert type(num_generation) is int and type(population_size) is int
        assert (0 <= mutation_chance <= 1) and (1 <= num_generation) and (1 <= patience)
        population = []
        expander = self._population_expander(population_size, mutation_chance)

        for _ in range(population_size):
            p = self._get_random_path()
            population.append(Path(p, self._graph))
        best_individual = max(population)

        count_generation = 0
        patience_current = 0
        while count_generation < num_generation and patience_current < patience:
            population = expander(population)
            next_gen_best_individual = max(population)
            if next_gen_best_individual > best_individual:
                best_individual = next_gen_best_individual
                patience_current = 0
            else:
                patience_current += 1
            count_generation += 1
        return best_individual

In [5]:
@anvil.server.callable
def genetic_search(cities_input, matrix_input, num_gen_input, popusize_input, mutate_input, patience_input):
    num_city_error = 0
    cities = [city.strip() for city in cities_input.split(',')]
    matrix = []
    rows = matrix_input.strip().split("\n")
    for row in rows:
        distances = [float(distance) for distance in row.split()]
        if len(distances) != len(cities):
            num_city_error = 1
            break
        matrix.append(distances)
    if len(rows) != len(cities):
        num_city_error = 1

    num_gen_ = num_gen_input
    popu_size = popusize_input
    mutate = mutate_input
    patience = patience_input

    if len(cities) != len(matrix):
        num_city_error = 1

    num_gen_error = 0
    if num_gen_input < 1:
        num_gen_error = 1

    popusize_error = 0
    if popusize_input < 1:
        popusize_error = 1

    mutate_error = 0
    if mutate_input > 1 or mutate_input < 0:
        mutate_error = 1

    patience_error = 0
    if patience_input < 1:
        patience_error = 1

    errors = [num_city_error, num_gen_error, popusize_error, mutate_error, patience_error]
    errors = np.nonzero(errors)[0].tolist()

    result = None
    if len(errors) == 0:
        graph = {}
        for i in range(len(matrix)):
            city_begin = cities[i]
            city_dict = {}
            for j in range(len(matrix[i])):
                city_end = cities[j]
                city_dict[city_end] = matrix[i][j]
            graph[city_begin] = city_dict

        searcher = GeneticSearch(graph)
        result = searcher.search(num_generation=num_gen_input,
                                 population_size=popusize_input,
                                 mutation_chance=mutate_input,
                                 patience=patience_input)
        result = (', '.join(result.path()), 1/result.value())
    return result
anvil.server.wait_forever()

KeyboardInterrupt: ignored

### Write code

In [ ]:
%%writefile traveller.py
import numpy as np
from copy import deepcopy



class TravellerProblem(object):
    def __init__(self, graph):
        self.graph = graph
        self.count_city = len(graph)

    def get_random_path(self):
        """
        """
        is_visited_city = dict(
            (city, False)
            for city in self.graph.keys()
        )
        start_city = np.random.choice(list(self.graph), 1)[0]
        is_visited_city[start_city] = True
        path = []

        while len(path) < self.count_city - 1:
            city_random = np.random.choice(list(self.graph), 1)[0] #chọn tp ngẫu nhiên

            if not is_visited_city[city_random]:
                is_visited_city[city_random] = True
                path += [city_random]
        return [start_city] + path + [start_city]

    def value(self, path):
        '''Returns the value of `state` as it is needed by optimization
           problems.
           Value is a number (integer or floating point).'''
        assert type(path) is list
        cost = 0
        for idx, city in enumerate(path):
            if idx == len(path) - 1:
                break
            next_city = path[idx + 1]
            cost += self.graph[city][next_city]
        return -cost


    def state_representation(self, path):
        """
        Returns a string representation of a state.
        By default it returns str(state).
        """
        return str(path)

class Path(object):
    def __init__(self, path, problem):
        assert type(path) is list and type(problem) is TravellerProblem
        self._path = path
        self._problem = problem
        self._value = problem.value(path)

    def value(self):
        return self._value

    def path(self):
        return self._path

    def __gt__(self, other):
        return self._value > other.value()

    def __add__(self, other):
        """
        'Crossover method' for genetic search. It should return a new state that
        is the 'mix' (somehow) of `state1` and `state2`.
        """
        assert type(other) is Path

        half_path1 = deepcopy(self._path[:len(self._path)//2])

        half_path2 = deepcopy(other.path())
        half_path2 = list(filter(lambda city: city not in half_path1, half_path2))

        return Path(half_path1 + half_path2, self._problem)

    def mutate(self):
        middle = len(self._path)//2
        pos_1 = np.random.randint(0, middle)
        pos_2 = np.random.randint(middle, len(self._path))
        self._path[pos_1], self._path[pos_2] = self._path[pos_2], self._path[pos_1]
        self._value = self._problem.value(self._path)

class GeneticSearch(object):
    def __init__(self, problem):
        assert type(problem) is TravellerProblem
        self.problem = problem

    def _get_random_parents(self, population):
        population_values = [1.1**(path.value()) for path in population]
        total_values = sum(population_values)
        '''xác suất của mỗi cá thể trong quần thể, cá thể nào
           chiếm càng lớn trong tổng value thì xác suất càng cao'''
        population_probs = [value/total_values for value in population_values]

        ''' method that choose father and mother.
            cá thể nào có value chiếm phần lớn trong tổng value
            của quần thể thì tỉ lệ được chọn làm cha mẹ càng cao'''
        return np.random.choice(population, 2, p=population_probs, replace=False)

    def _population_expander(self, population_size, mutation_chance, keep_old_generation):
        def expander(population):
            new_generation = []
            for _ in range(population_size):
                parents = self._get_random_parents(population)
                father, mother = parents[0], parents[1]
                child = father + mother

                '''random tỉ lệ child là đột biến hay không'''
                is_mutant = np.random.choice([True, False], 1, p=[mutation_chance, 1-mutation_chance])[0]
                if is_mutant:
                    child.mutate()
                new_generation.append(child)

            if not keep_old_generation:
                population.clear()
            else:
                population = population[:keep_old_generation]

            for next_gen in new_generation:
                population.append(next_gen)
            population.sort(reverse=True)

            population = population[:population_size]

        return expander

    def search(self, num_generation=100, population_size=100, mutation_chance=0.1, keep_old_generation=0):
        assert type(num_generation) is int and type(population_size) is int
        assert (0 <= mutation_chance <= 1) and type(keep_old_generation) is int and keep_old_generation <= num_generation

        population = []
        expander = self._population_expander(population_size, mutation_chance, keep_old_generation)
        count_generation = 0

        for _ in range(population_size):
            p = self.problem.get_random_path()
            population.append(Path(p, self.problem))

        count_generation += 1
        population.sort(reverse=True)
        best_individual = population[0]

        while count_generation < num_generation:
            expander(population)
            next_gen_best_individual = population[0]
            if next_gen_best_individual > best_individual:
                best_individual = next_gen_best_individual
            count_generation += 1
        return best_individual

Writing traveller.py


In [ ]:
%cat traveller.py

import numpy as np
from copy import deepcopy



class TravellerProblem(object):
    def __init__(self, graph, start_city):
        self.graph = graph
        self.start_city = start_city
        self.count_city = len(graph)

    def generate_random_path(self):
        """
        """
        is_visited_city = dict(
            (city, False)
            for city in self.graph.keys()
        )
        is_visited_city[self.start_city] = True
        path = []

        while len(path) < self.count_city - 1:
            city_random = np.random.choice(list(self.graph), 1)[0] #chọn tp ngẫu nhiên

            if not is_visited_city[city_random]:
                is_visited_city[city_random] = True
                path += [city_random]

        return path

    def value(self, path):
        '''Returns the value of `state` as it is needed by optimization
           problems.
           Value is a number (integer or floating point).'''
        assert type(path) is list

        next_city = path[0